In [ ]:
import tensorflow.keras as keras
import numpy as np
import os
import PIL
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

### Load the data

In [ ]:
indata = keras.datasets.mnist.load_data()

In [ ]:
len(indata[1][1])

### Split the data into a training set and a test set

In [ ]:
training_data = indata[0]
test_data = indata[1]


### Create the neural network model that we'll use

In [ ]:
input_layer = keras.layers.Input(shape=(28, 28, 1))
model = keras.layers.Conv2D(32, (3,3))(input_layer)
model = keras.layers.MaxPooling2D()(model)
model = keras.layers.Conv2D(64, (3,3))(model)
model = keras.layers.MaxPooling2D()(model)
model = keras.layers.Flatten()(model)
model = keras.layers.Dense(128)(model)
model = keras.layers.Dropout(0.5)(model)
model = keras.layers.Dense(10, activation='softmax')(model)

model = keras.models.Model(inputs=[input_layer], outputs=[model])

In [ ]:
adam = keras.optimizers.Adam()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer=adam,
             metrics=['accuracy'],
             )

### Look at the model structure

In [ ]:
model.summary()

### Train the model 
Feed the images to the network and train it 

In [ ]:
model.fit(training_data[:].reshape(-1, 28, 28, 1), training_data[1][:],
         epochs=5)

In [ ]:
training_data[1][:10]

### Now test our model on the set of data that was held out
How well does it do with images it has never seen before?

In [ ]:
Y_pred = model.predict(test_data[0][:].reshape(-1, 28, 28, 1)/255.)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
print(classification_report(test_data[1][:], y_pred))

### Mount the Google Drive with the edited images

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
indir = '/content/drive/MyDrive/numbers/2/'
student_photos = []
student_photos_arrays = []

for infile in os.listdir(indir):
    if infile[-3:] == 'png':
        student_photos.append(PIL.Image.open(indir+infile))
        student_photos_arrays.append(np.array(PIL.Image.open(indir+infile)))
        

How many images of the number 2 are there?

In [ ]:
len(student_photos)

### Take a look at the original images

In [ ]:
student_photos[1]

### The 'raw' data comprising the image

In [ ]:
np.array(student_photos[0])

In [ ]:
student_photos_arrays[0].shape

In [ ]:
temp_input = np.array(student_photos[0].resize((28,28))).reshape(-1, 28, 28, 1)
temp_input[temp_input < 0.1] = 0
model.predict(temp_input)

What happened here? 4 outputs with only 1 image input?

### What does our original data look like?

In [ ]:
print('Original data:', training_data[0].shape)
print('Our data:', student_photos_arrays[0].shape)

### What about the actual data itself?

In [ ]:
training_data[0]

In [ ]:
student_photos_arrays[0]

### What's different about these two?
Will that impact our model and our predictions?

### Let's try out our data

In [ ]:
image_size = 28
indir = '/content/drive/MyDrive/numbers/'

def edit_photo(inphoto):
    out = inphoto - 255
    out *= -1
    out /= 255.
    #out[out < 0.1] = 0.0
    return out
    
input_photos = keras.preprocessing.image.ImageDataGenerator(preprocessing_function = edit_photo)
photos = input_photos.flow_from_directory(indir, 
                                          shuffle=False,
                                          color_mode='grayscale', #all ifcb images are grayscale
                                          class_mode='sparse', #there are multiple classes of images (i.e. > 2)
                                          target_size=(image_size,image_size),  #squish/stretch images to this size
                                          batch_size=1)

### Here's a look at our input data

In [ ]:
plt.imshow(photos.next()[0][0].reshape(28,28), cmap='gray_r')

In [ ]:
photos.reset()
predictions = model.predict_generator(photos)

In [ ]:
for idx, indiv_number in enumerate(predictions):
    print(np.argmax(indiv_number), photos.classes[idx], indiv_number)

In [ ]:
student_photos[0].resize((28,28))

In [ ]:
plt.imshow(temp_input.reshape(28, 28))

In [ ]:
fig = plt.figure()
fig = plt.subplots(22, 4, figsize=(15,55))
photos.reset()

for idx, indiv_number in enumerate(predictions):
    plt.subplot(22,4,idx+1)
    temp = photos.next()
    plt.imshow(temp[0].reshape(28,28), cmap = 'gray_r')
    plt.annotate('{}, {}'.format(np.argmax(indiv_number), photos.classes[idx]), (1,4))
    plt.annotate('{:2.2f}'.format(indiv_number[np.argmax(indiv_number)]), (1,27))
    
plt.tight_layout()

### Reduce the learning rate

In [ ]:
model.optimizer.lr = 0.0002

### Add some noise to the training set to mimic our images

In [ ]:
#add some noise to the training images
noisy_training_data = training_data[0].copy()/255.
for idx, x, in enumerate(noisy_training_data):
    noise = np.random.rand(28*28).reshape(28, 28) / 2.
    noisy_training_data[idx] += noise
    

Here's what the noisy data look like:

In [ ]:
plt.imshow(noisy_training_data[105], cmap='gray_r')

And here's the original data:

In [ ]:
plt.imshow(training_data[105], cmap='gray_r')

In [ ]:
model.fit(noisy_training_data[:].reshape(-1, 28, 28, 1), training_data[1][:],
         epochs=1)

### Try training with a few of our images
Does it help?

In [ ]:
temp = photos.next()
temp_num = temp[0][0]
temp_target = temp[1]
plt.imshow(temp_num.reshape(28,28))

In [ ]:
#went through and made sure that 9 was next

model.fit(temp_num.reshape(-1, 28, 28, 1), np.array([temp_target]), epochs = 1)

In [ ]:
photos.reset()
predictions = model.predict_generator(photos)

In [ ]:
fig = plt.figure()
fig = plt.subplots(22, 4, figsize=(15,55))
photos.reset()

for idx, indiv_number in enumerate(predictions):
    plt.subplot(22,4,idx+1)
    temp = photos.next()
    plt.imshow(temp[0].reshape(28,28), cmap = 'gray_r')
    plt.annotate('{}, {}'.format(np.argmax(indiv_number), photos.classes[idx]), (1,4))
    plt.annotate('{:2.2f}'.format(indiv_number[np.argmax(indiv_number)]), (1,27))
    
plt.tight_layout()